In [1]:
import gym
import random, math
import numpy as np
import arcade
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from PIL import Image

        
# from LightEnvCopy import LightEnv
from LightEnvCopy import LightEnv

import gym.spaces
from gym.spaces import Discrete, Box

from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog

from collections import namedtuple

# Do the math to figure out our screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
SCREEN_TITLE = "Game 1: Let There Be Light!"

SPRITE_SCALING = 0.25

# How fast the camera pans to the player. 1.0 is instant.
CAMERA_SPEED = 0.1

PLAYER_MOVEMENT_SPEED = 7
BOMB_COUNT = 5
TORCH_COUNT = 1
PLAYING_FIELD_WIDTH = 800 #1600
PLAYING_FIELD_HEIGHT = 600 #1600
REWARD_COUNT = 1 #TF - Add in reward
END_GAME = False
torch_collected = False

#TF Start - Adding in actions for action conversion

# COnvenient data structure to hold information about actions
Action = namedtuple('Action', 'name index delta_i delta_j')

up = Action('up', 0, -1, 0)    
down = Action('down', 1, 1, 0)    
left = Action('left', 2, 0, -1)    
right = Action('right', 3, 0, 1)    

index_to_actions = {}
for action in [up, down, left, right]:
    index_to_actions[action.index] = action
# print(index_to_actions[0].name)
str_to_actions = {}
for action in [up, down, left, right]:
    str_to_actions[action.name] = action
#TF End - Adding in actions for action conversion


class LightEnvWrapper(gym.Env, LightEnv):
    """Class that wraps the Lights Environment to make it 
    compatible with RLLib."""

    metadata = {"render.modes": ["rgb_array", "state_pixels"]}
    
    def __init__(self, config: EnvContext):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
        self.counting = 0
        
        self.torch_collected = False
        self.torch_collected_count = []
        
        self.mygame = LightEnv
        #The action space is a choice of 9 actions: U/D/L/R/UR/DR/DL/UL/DO NOTHING. Not continuous
        #because speed of agent is fixed. Or potentially just 4: U/D/L/R.
        self.action_space = Discrete(4)
#         self.action_space = config.get("parrot_shriek_range", Discrete(4))
        #The observation space is a fixed image of the current game screen - fully observable.
        #Can set to a view just around the player using arcade.set_viewport.
        #Need obs space to be either 42,42,x or 84,84,x to be compatible with rllib.
        self.observation_space = Box(low=0, high=255, shape=(84,84, 4), dtype=np.uint8)
        
        self.counting = 0

    def reset(self):
        print("resetting in wrapper")
        
        if self.torch_collected == 1:
            print("Torch was collected this episode!")
        else:
            print("Torch was not collected this episode...")
        self.torch_collected_count.append(self.torch_collected)
        print(self.torch_collected_count)

        
        self.render(drawing=True)
        #Resets the state of the environment for a new episode and an initial observation.
        obs_mygame = self.mygame.reset(self)
        
        #Open up the resetted image to verify working correctly.
        obs_mygame.show()
        
        #Convert observation to 84x84 resolution and np array for rllib.
        obs = self.convert_observations(obs_mygame)
        
#         print("resetted")
        return obs

    def step(self, action):
        self.counting += 1
        
        #Making sure an action is chosen, either: 0, 1, 2, 3.
        assert action in [0, 1, 2, 3] #0-up,1-down,2-left,3-right.
        
        #Convert the numeric action to a keyword: up, down, left, right.
        actions_myenv = index_to_actions[action].name #returns a word, one of: up/down/left/right
#         print(f"action taken: {actions_myenv}")
        
        #Update the window with on_update()
        self.render(self)
        
        #Compute observation extracted from the window (800x600), with reward and done flag.
        obs, reward, done, torch_collected = self.mygame.step(self,actions_myenv)
        
        if torch_collected == True:
            self.torch_collected = 1
        else:
            self.torch_collected = 0

        if self.counting % 33 == 0: #33 steps roughly equates to 1 second in game time
            print(f"total score is {self.score} at time: {self.mygame.time_taken_reported(self)}")
        
        #Convert observation to 84x84 resolution and np array for rllib.
        obs_mygame = self.convert_observations(obs)
        
        #If the reward has been obtained, reset the environment and start again
        if done == True:
            print(f"done is {done}, resetting environment in wrapper.")
            self.reset()
        
        return obs_mygame, reward, done, {}

    def seed(self, seed=None):
        random.seed(seed)

    def convert_observations(self, obs_mygame): #resizing and converting to array for rllib processing
        # We normalize and concatenate observations
        obs = obs_mygame
        obs_resized = obs.resize((84,84))
        obsarray = np.array(obs_resized)
        return obsarray
    
    def render(self, mode='state_pixels', drawing=False):
        self.mygame.on_update(self, 1/60)
        self.mygame.on_draw(self)
        test = self.mygame.time_taken_reported(self)
        

C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\tensorflow_probability\python\__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


### Now run the rllib script to train the agent

In [3]:
import ray.rllib.algorithms.sac.sac as sac
from ray.rllib.algorithms.sac.sac import SACConfig

config = SACConfig().training(gamma=0.9, lr=0.01)\
    .resources(num_gpus=0)\
    .rollouts(num_rollout_workers=1, recreate_failed_workers=True)
config.replay_buffer_config['capacity']=100000
config.optimization['entropy_learning_rate']=0.05
config.env=LightEnvWrapper
print(config.to_dict())
# Build a Algorithm object from the config and run 1 training iteration.
# trainer = config.build(env=LightEnvWrapper)
trainer = sac.SAC(config=config)


avg_rewards = []
num_iterations = []
time_spent = []
for episode in range(1000):
    print("Starting episode ", episode)
    # Perform one iteration of training the policy with PPO
    result = trainer.train()
    #print(pretty_print(result))
    print("episode reward mean: ", result['episode_reward_mean'])
    avg_rewards.append(result['episode_reward_mean'])
    num_iterations.append(episode)
#     if episode % 10 == 0:
#         checkpoint = trainer.save()
#         print("checkpoint saved at", checkpoint)
    print("End of episode ", episode)



2022-09-16 12:04:29,798	WARNING deprecation.py:47 -- DeprecationWarning: `config['multiagent']['replay_mode']` has been deprecated. config['replay_buffer_config']['replay_mode'] This will raise an error in the future!


{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': <class '__main__.LightEnvWrapper'>, 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': False, 'num_workers': 1, 'num_envs_per_worker': 1, 'sample_collector': <class 'ray.rllib.evaluation.collectors.simple_list_collector.SimpleListCollector

(pid=18544) Windows fatal exception: code 0xc0000139
(pid=18544) 
(pid=18544) C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\tensorflow_probability\python\__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18544)   if (distutils.version.LooseVersion(tf.__version__) <
(RolloutWorker pid=18544) 2022-09-16 12:04:46,038	WARNING env.py:142 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


(RolloutWorker pid=18544) resetting in wrapper
(RolloutWorker pid=18544) Torch was not collected this episode...
(RolloutWorker pid=18544) [False]
(RolloutWorker pid=18544) resetting


2022-09-16 12:04:54,088	INFO trainable.py:160 -- Trainable.setup took 24.292 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-09-16 12:04:54,094	WARNING util.py:65 -- Install gputil for GPU system monitoring.


Starting episode  0
(RolloutWorker pid=18544) resetting in wrapper
(RolloutWorker pid=18544) Torch was not collected this episode...
(RolloutWorker pid=18544) [False, 0]
(RolloutWorker pid=18544) resetting


2022-09-16 12:04:57,409	ERROR algorithm.py:2173 -- Error in training or evaluation attempt! Trying to recover.
Traceback (most recent call last):
  File "C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\ray\rllib\algorithms\algorithm.py", line 2373, in _run_one_training_iteration
    results = self.training_step()
  File "C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\ray\rllib\algorithms\dqn\dqn.py", line 358, in training_step
    new_sample_batch = synchronous_parallel_sample(
  File "C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\ray\rllib\execution\rollout_ops.py", line 100, in synchronous_parallel_sample
    sample_batches = ray.get(
  File "C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\ray\_private\client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\ray\_private\worker.py", line 2275, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RayOutOfMemoryError): 

(pid=5412) Windows fatal exception: code 0xc0000139
(pid=5412) 
(pid=5412) C:\Users\Tim\Anaconda3\envs\rllib\lib\site-packages\tensorflow_probability\python\__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=5412)   if (distutils.version.LooseVersion(tf.__version__) <
(RolloutWorker pid=5412) 2022-09-16 12:05:11,058	WARNING env.py:142 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


(RolloutWorker pid=5412) resetting in wrapper
(RolloutWorker pid=5412) Torch was not collected this episode...
(RolloutWorker pid=5412) [False]
(RolloutWorker pid=5412) resetting
(RolloutWorker pid=5412) resetting in wrapper
(RolloutWorker pid=5412) Torch was not collected this episode...
(RolloutWorker pid=5412) [False, 0]
(RolloutWorker pid=5412) resetting
(RolloutWorker pid=5412) total score is -1 at time: 1
(RolloutWorker pid=5412) total score is -1 at time: 1
(RolloutWorker pid=5412) total score is -2 at time: 2
episode reward mean:  nan
End of episode  0
Starting episode  1
(RolloutWorker pid=5412) total score is -2 at time: 2
(RolloutWorker pid=5412) total score is -3 at time: 3
(RolloutWorker pid=5412) total score is -3 at time: 3
episode reward mean:  nan
End of episode  1
Starting episode  2
(RolloutWorker pid=5412) total score is -4 at time: 4
(RolloutWorker pid=5412) total score is -4 at time: 4
(RolloutWorker pid=5412) total score is -5 at time: 5
episode reward mean:  nan

KeyboardInterrupt: 